In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [2]:
# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.common.by import By

In [3]:
# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

Fase 2: Extracción de Detalles de Películas con Selenium
Una vez que tengan una lista de películas de la API, el siguiente paso es obtener información más detallada sobre ellas. Utilizaras la herramienta de automatización web Selenium para navegar a sitios web de reseñas de películas (IMDB y Rotten Tomatoes) y extraer detalles como calificaciones, actores y directores. En concreto deberás extraer información sobre:

Puntuacion de IMDB (en caso de que la tenga).

Puntuación de Rotten Tomatoes (Tomatometer).

Dirección (director/a o directore/as de cada película).

Gionistas (de cada película).

Argumento.

Duración (en minutos).

Nombre de la película

In [4]:
# # Llamo al navegador

# driver=webdriver.Chrome()
# driver.get("https://www.imdb.com/?ref_=nv_home")
# driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()

# # Hago grande el navegador para que aparezcan todas las opciones posibles de la web

# driver.maximize_window()

# sleep(5)


def llamar_info_peli(lista_id_muestra,driver):
  # Creo lista de tuplas tal y como pide el enunciado para guardar los datos extraidos 

  lista_tuplas_pelis=[]

  # Creamos una muestras de id de peliculas por las que vamos a iterar para extraer la info

  lista_id_muestra= ['tt10579986','tt10597228','tt11097384','tt11152168','tt11237950']


  for id in lista_id_muestra:
      
      # Creo la lista donde voy a ir guardando la info de cada peli, que despues convertiremos en tupla 

      lista_datos_imdb=[]

      try:

        busqueda = driver.find_element("css selector","#suggestion-search")

        busqueda.clear()

        busqueda.send_keys(id, Keys.ENTER)

        # Extraemos la informacion que necesitamos

        nombre_pelicula=driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[1]/h1/span').text
        #nombre_pelicula=driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(5) > section > section > div.sc-92625f35-3.frxYSZ > div.sc-b7c53eda-0.dUpRPQ > h1 > span").text 
      
        lista_datos_imdb.append(nombre_pelicula)


        # DIRECCION: extraemos la informacion de los directores, que pueden ser 1 o varios
        # Creamos una lista donde iremos agregando los diferentes directores en base a las etiquetas correspondientes

        #direccion_peli=[]

        
        try: 
          
          director1= driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[2]/ul/li[1]/div/ul/li[1]/a').text
          lista_datos_imdb.append(director1) 

        except:      
            director1= driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[3]/ul/li[1]/div/ul/li/a').text
            lista_datos_imdb.append(director1)

                                            
        # Guionistas

        try:
          
          guionistas_peli= driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[2]/ul/li[2]/div').text
          guionistas_bien=re.sub(r'([a-z])([A-Z])', r'\1 \2', guionistas_peli)
          lista_datos_imdb.append(guionistas_bien)

        except:
            guionistas_peli= driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[3]/ul/li[2]/div').text
            guionistas_bien=re.sub(r'([a-z])([A-Z])', r'\1 \2', guionistas_peli)
            lista_datos_imdb.append(guionistas_bien)

        # Para argumento hay varias etiquetas, una para cada tamaño de pantalla o dispositivo, por lo que hay que escoger la de pantalla grande
        argumento_peli=driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/p/span[3]').text
        lista_datos_imdb.append(argumento_peli) 

      
        # Duracion pelicula:

        try: 
          duracion_peli=driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[1]/ul/li[3]').text
          #duracion_peli= driver.find_element("css selector","__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(28) > div.sc-f65f65be-0.bBlII > ul > li:nth-child(1) > div").text
          lista_datos_imdb.append(duracion_peli)

        except:

          try:  
            duracion_peli=driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[1]/ul/li[2]').text
            lista_datos_imdb.append(duracion_peli)
            
          except:
            lista_datos_imdb.append("No encontrado")
          
    
        # Puntuacion
        try:
          
          puntuacion_imdb= driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[2]/div/div[1]/a/span/div/div[2]/div[1]/span[1]').text
          lista_datos_imdb.append(puntuacion_imdb.replace(",", "."))

        except:
        
          lista_datos_imdb.append("No encontrado")

      except Exception as null:
          print(f"Error al procesar la búsqueda para el ID {id}: {str(null)}")

    
    # Guardo todos los datos extraidos de cada peli en una tupla, que se incluye, a su vez, en la lista de tuplas de todas las peliculas

      lista_tuplas_pelis.append(tuple(lista_datos_imdb))


      # Sale a pagina principal y vuelve a empezar bucle

      driver.back()

  return lista_tuplas_pelis

In [26]:
#lista_tuplas_pelis

[("Mom's Coming",
  'Rishiking',
  'Rishiking',
  'Un alma supuestamente tiene como hija a una chica de 20 años, a la que quiere llevarse con ella, y la hermana mayor quiere impedir que el alma lo haga.',
  '2h 2min',
  'No encontrado'),
 ('Love Song and Power',
  'Erik Krefeld',
  'Stevie Kinchen Erik Krefeld',
  'Esta es una historia sobre encontrar el valor para ser uno mismo.',
  '1h 30min',
  'No encontrado'),
 ('El astronauta',
  'Johan Renck',
  'Jaroslav Kalfar Colby Day',
  'Jakub Procházka, huérfano y criado en el campo por sus abuelos, supera todas las adversidades y se convierte en el primer astronauta del país.',
  '1h 47min',
  '5.7'),
 ('Amigos imaginarios',
  'John Krasinski',
  'John Krasinski',
  'Sigue a una niña que pasa por una experiencia difícil y entonces empieza a ver a los amigos imaginarios de todo el mundo que se han quedado atrás cuando sus amigos de la vida real han crecido.',
  '1h 44min',
  '8.0'),
 ('The Accompanist Awakening',
  'Frederick Keeve',
  'F

In [5]:
# Llamo al navegador

driver=webdriver.Chrome()
driver.get("https://www.imdb.com/?ref_=nv_home")
driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()

# Hago grande el navegador para que aparezcan todas las opciones posibles de la web

driver.maximize_window()

sleep(5) 

lista_id_muestra= ['tt10579986','tt10597228','tt11097384','tt11152168','tt11237950']
resultados_muestra=llamar_info_peli(lista_id_muestra,driver)

In [6]:
resultados_muestra

[("Mom's Coming",
  'Rishiking',
  'Rishiking',
  'Un alma supuestamente tiene como hija a una chica de 20 años, a la que quiere llevarse con ella, y la hermana mayor quiere impedir que el alma lo haga.',
  '2h 2min',
  'No encontrado'),
 ('Love Song and Power',
  'Erik Krefeld',
  'Stevie Kinchen Erik Krefeld',
  'Esta es una historia sobre encontrar el valor para ser uno mismo.',
  '1h 30min',
  'No encontrado'),
 ('El astronauta',
  'Johan Renck',
  'Jaroslav Kalfar Colby Day',
  'Jakub Procházka, huérfano y criado en el campo por sus abuelos, supera todas las adversidades y se convierte en el primer astronauta del país.',
  '1h 47min',
  '5.7'),
 ('Amigos imaginarios',
  'John Krasinski',
  'John Krasinski',
  'Sigue a una niña que pasa por una experiencia difícil y entonces empieza a ver a los amigos imaginarios de todo el mundo que se han quedado atrás cuando sus amigos de la vida real han crecido.',
  '1h 44min',
  '7.6'),
 ('The Accompanist Awakening',
  'Frederick Keeve',
  'F

In [ ]:
#datos_csv=pd.read_csv(r'...)

In [ ]:
# id_comedia=datos_csv["ID"]
# id_comedia

In [ ]:
#lista_id_comedia=id_comedia.tolist()
# lista_id_comedia